# Машинное обучение, ФКН ВШЭ

## Практическое задание 10. Рекомендательные системы

### Общая информация

Дата выдачи: 09.05.2020  

Мягкий дедлайн: 23:59MSK 24.05.2020 (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 27.05.2020


### О задании
В этом задании мы будем практиковаться в построении рекомендательных систем: научимся считать метрики и пообучаем модели.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов. Проверяющий имеет право снизить оценку за неэффективную реализацию или неопрятные графики.

**Обратите внимание**, что в каждом разделе домашнего задания есть оцениваниемые задачи и есть вопросы. Вопросы дополняют задачи и направлены на то, чтобы проинтерпретировать или обосновать происходящее. Код без интерпретации не имеет смысла, поэтому отвечать на вопросы обязательно — за отсутствие ответов мы будем снижать баллы за задачи. Если вы ответите на вопросы, но не напишете корректный код к соответствующим оцениваемым задачам, то баллы за такое выставлены не будут.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. 

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tqdm

### Данные

Будем работать с датасетом  от онлайн-кинотеатра Okko [REKKO CHALLENGE](https://boosters.pro/championship/rekko_challenge/data).

Имеющиеся данные содержат несколько файлов. Рассмотрим всё, что у нас есть.

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах:

In [0]:
catalogue = pd.read_json('data/catalogue.json').T

In [0]:
catalogue['element_uid'] = catalogue.index

In [0]:
catalogue.head()

,attributes,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,type,element_uid
0,"[31115, 6713, 10906, 31116, 31117, 270, 24431,...",[],80,2.9122e+07,0.57526,0,1.12833,0.654707,movie,0
1,"[2786, 385, 2799, 3730, 886, 7, 11700, 42, 20,...","[purchase, rent]",120,6.61043e+06,0.773224,3,1.11201,0.654707,movie,1
10,"[10511, 2752, 9076, 1373, 472, 7, 10512, 42, 4...",[],100,1.66661e+07,0.700997,0,1.13008,0.654707,movie,10
100,"[18441, 16300, 16580, 18770, 18771, 18643, 396...","[purchase, rent]",80,6.06474e+06,0.752751,4,0.95371,0,movie,100
1000,"[1256, 441, 19806, 15224, 16708, 83, 336, 1980...",[],80,9.11502e+06,0.689883,0,1.12833,0.68041,movie,1000


 - `attributes` — мешок атрибутов;
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`);
 - `duration` — длительность фильма (для сериалов и многосерийных фильмов — длительность серии) в минутах, округлённая до десятков;
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков;
 - `type` — принимает значения `movie`, `multipart_movie` или `series`.

`transactions.csv` — список всех транзакций за определённый период времени:

In [0]:
transactions = pd.read_csv('data/transactions.csv',
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [0]:
transactions.head()

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50
3,6272,74296,S,4.430518e+07,2530,0,99
4,5543,340623,P,4.430518e+07,6282,0,50


In [0]:
transactions.shape

(9643012, 7)

 - `element_uid` — идентификатор элемента (фильма);
 - `user_uid` — идентификатор пользователя;
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке);
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке;
 - `watched_time` — число просмотренных по транзакции секунд;
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр;
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр.

`ratings.csv` содержит информацию о поставленных пользователями оценках:

In [0]:
ratings = pd.read_csv('data/ratings.csv')

In [0]:
ratings.head()

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07


In [0]:
ratings.shape

(438790, 4)

 - `rating` — поставленный пользователем рейтинг (от `0` до `10`).

Далее будем обучать модель на долю времени просмотра фильма. Для этого к матрице `transactions` добавим информацию о длительности фильма.

In [0]:
transactions = transactions.merge(catalogue, how='left', on='element_uid')

 В колонке `transactions['duration']` могут быть нули, так как значения округляются до ближайшей десятки. Поэтому добавим 10 ко всем значениям. И умножим все значения `transactions['duration']` на 60, чтобы перевести в секунды. 

In [0]:
transactions.loc[:, 'duration'] += 10
transactions['duration']  = transactions['duration']*60

Для простоты вычислений далее будем использовать не всех пользователей.

In [0]:
transactions_small = transactions.loc[transactions.user_uid < transactions.user_uid.quantile(.01)]

In [0]:
transactions.element_uid.nunique()

8296

Чтобы оценивать качество предсказаний построенных алгоритмов, разобьем данные на обучающую и тестовую выборки. Для этого вычислим 75-й перцентиль признака `ts` и разделим данные `transactions_small` во времени, так, чтобы в обучающую выборку попало 75% данных, а в тестовую — остальные 25%.

In [0]:
transactions_test = transactions_small.loc[transactions_small.ts > transactions_small.ts.quantile(.75)].reset_index()
transactions_train = transactions_small.loc[transactions_small.ts < transactions_small.ts.quantile(.75)].reset_index()

In [0]:
transactions_train.shape

(72319, 17)

In [0]:
transactions_train.user_uid.nunique()

4498

In [0]:
transactions_test.user_uid.nunique()

2733

Данные `ratings` тоже необходимо сократить и исключить оценки из тестовой части:

In [0]:
test_user_ids = set(transactions_test[['element_uid', 'user_uid']].itertuples(index=False))
train_user_ids = set(transactions_train[['element_uid', 'user_uid']].itertuples(index=False))
drop_index = []
for row in tqdm.tqdm(ratings.itertuples()):
    if (row.element_uid, row.user_uid) in test_user_ids or not (row.element_uid, row.user_uid) in train_user_ids:
        drop_index.append(row.Index)

438790it [00:01, 331779.01it/s]


In [0]:
ratings.drop(drop_index, inplace=True)
ratings.reset_index(inplace=True)

In [0]:
ratings.shape

(2836, 5)

На выделенной обучающей выборке необходимо обучить рекомендательную систему и предсказать top-20 наиболее релевантных для пользователя идентификаторов контента.

Получим значение целевой переменной для итогового алгоритма — долю времени просмотра фильма:

In [0]:
transactions_train['target'] = transactions_train.watched_time/transactions_train.duration

In [0]:
transactions_test['target'] = transactions_test.watched_time/transactions_test.duration

В поле `target` присутствуют значения больше 1, так как для сериалов и многосерийных фильмов используется средняя длина серии — заменим в таких случаях целевую переменную на 1:

In [0]:
transactions_train.loc[transactions_train['target'] > 1, 'target'] = 1
transactions_test.loc[transactions_test['target'] > 1, 'target'] = 1

Далее в задании везде будем обозначать как $r_{ui}$ значение этой целевой переменной для пользователя $u$ и элемента $i$.

### Задание 0 [0.5 балла]

Для оценки качества предсказаний будем использовать метрику $MNAP@20$, которую использовали в соревновании.
Метрика отличается от оригинальной $MAP$ тем, что значение для каждого пользователя нормализуется не на константу, а на число потреблённых фильмов. Таким образом, вес угадывания одного фильма больше у пользователей, имеющих меньшее число просмотров.


$$MNAP@20 = \dfrac{1}{|U|}\sum_{u\in U}\dfrac{1}{\min(n_u, 20)}\sum_{i=1}^{20}r_u(i)p_u@i $$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^kr_u(j)$$
- $r_{u}(i)$ — потребил ли пользователь u контент, предсказанный ему на месте i (1 либо 0);
- $n_u$ — количество элементов, которые пользователь потребил за тестовый период;
- $U$ — множество тестовых пользователей.

Реализуйте функцию `mnap_k`, вычисляющую метрику качества $MNAP@K$, где

- `predictions` — предсказания фильмов для пользователей  `List[List[int]]`;

- `target` — фильмы которые посмотрел пользователь  `List[Set[int]]`.

In [0]:
def mnap_k(predictions, target, k=20):
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・
    return


Проведем элементарные проверки на корректность реализации:

In [0]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert mnap_k(test, target) == 1.0

In [0]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 11)), set(np.arange(2, 12)), set(np.arange(3, 13))]
assert mnap_k(test, target) == 1.0

In [0]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(2, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert round(mnap_k(test, target), 3) == 0.954

### Задание 1 [1.5 балла]



#### Memory-based
Два пользователя похожи, если им нравятся одинаковые фильмы.
Рассмотрим двух пользователей $u$ и $v$ и обозначим через $I_{uv}$ множество фильмов $i$, которые просмотрели пользователи:
$$
    I_{uv}
    =
    \{
        i \in I
        |
        \exists r_{ui}
        \ \&\ 
        \exists r_{vi}
    \}.
$$
Тогда сходство двух данных пользователей можно вычислить через корреляцию Пирсона:
$$
    w_{uv}
    =
    \frac{
        \sum_{i \in I_{uv}}
            (r_{ui} - \bar r_u)
            (r_{vi} - \bar r_v)
    }{
        \sqrt{
        \sum_{i \in I_{uv}}
            (r_{ui} - \bar r_u)^2
        }
        \sqrt{
        \sum_{i \in I_{uv}}
            (r_{vi} - \bar r_v)^2
        }
    },
$$
где $\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей.

Чтобы вычислять сходства между элементами $i$ и $j$, введём множество пользователей $U_{ij}$, для которых известны целевые переменные для этих элементов:
$$
    U_{ij}
    =
    \{
        u \in U
        |
        \exists r_{ui}
        \ \&\ 
        \exists r_{uj}
    \}.
$$
Тогда сходство двух данных товаров можно вычислить через корреляцию Пирсона:
$$
    w_{ij}
    =
    \frac{
        \sum_{u \in U_{ij}}
            (r_{ui} - \bar r_i)
            (r_{uj} - \bar r_j)
    }{
        \sqrt{
        \sum_{u \in U_{ij}}
            (r_{ui} - \bar r_i)^2
        }
        \sqrt{
        \sum_{u \in U_{ij}}
            (r_{uj} - \bar r_j)^2
        }
    },
$$
где $\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов

Реализуйте класс `MemoryBased` с методами:

- `__init__` — конструктор класса, принимает на вход матрицу целевых переменных $r_{ui}$;
- `user_similarity` — возвращает вектор $w_u$ сходства пользователя $u$ и всех пользователей из обучающей выборки; если пользователь $u$ не встречался в обучающей выборке, то возвращайте нулевой вектор;
- `item_similarity` — возвращает вектор $w_i$ сходства фильма $i$ и всех фильмов из обучающей выборки; если фильм $i$ не встречался в обучающей выборке, то возвращайте нулевой вектор.



Уделите особое внимание оптимизации кода. Используйте векторизацию и матричные вычисления, где это возможно.

In [0]:
class MemoryBased():
    def __init__(self, ratings):
        self.ratings = ratings
        pass
        
    def user_similarity(self, test_user):
      """test_user - вектор соответствующий пользователю"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass
        
    def item_similarity(self, test_item):
      """test_item - вектор соответствующий фильму"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass
        
    

Проведем элементарные проверки на корректность реализации:

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по всем просмотренным фильмам

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по всем пользователям, которые смотрели фильм

In [0]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1])
assert np.all(np.round(result, 2) == np.array([1.0, -0.94, 0.92, 0.97]))

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по множеству $I_{uv}$

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по множеству $I_{ij}$

In [ ]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1])
assert np.all(np.round(result, 2) == np.array([1.0, -1.0, 0.0, 1.0]))

### Задание 2 [1 балл]

#### User-Based
В подходе на основе сходств пользователей
определяется множество $U(u_0)$ пользователей, похожих на данного:
$$
    U(u_0)
    =
    \{v \in U
        |
        w_{u_0 v} > \alpha
    \}.
$$
После этого для каждого товара вычисляется, как часто он покупался пользователями из $U(u_0)$:
$$
    p_{i}
    =
    \frac{
        |\{u \in U(u_0) | \exists r_{ui}\}|
    }{
        |U(u_0)|
    }.
$$
Пользователю рекомендуются $k$ товаров с наибольшими значениями $p_i$.
Данный подход позволяет строить рекомендации, если для данного пользователя найдутся похожие.
Если же пользователь является нетипичным, то подобрать что-либо не получится.

Реализуйте класс `UserBased`, наследованный от `MemoryBased`. Считайте, что $\alpha = 0$.

Класс `UserBased` должен иметь метод реализованный `recomendation_k` с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [0]:
class UserBased(MemoryBased):
    def recomendation_k(self, user_vectors, k=20):
      """user_vectors - матрица размера N x M, где M - общее число фильмов, N - число пользователей, для которых делаем рекомендации"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass
    

Проведем элементарные проверки на корректность реализации:

In [0]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
item_based = UserBased(I)
result = item_based.recomendation_k(np.array([0.5, 0.4, 0, 0.1]), k=1)
assert np.all(result == np.array([2]))

### Задание 3 [1 балл]

#### Item-Based 

Определяется множество фильмов, похожих на те, которые интересовали данного пользователя:
$$
    I(u_0)
    =
    \{
        i \in I
        |
        \exists r_{u_0 i_0},
        w_{i_0 i} > \alpha
    \}.
$$
Затем для каждого товара из этого множества вычисляется его сходство с пользователем:
$$
    p_i
    =
    \max_{i_0: \exists r_{u_0 i_0}}
    w_{i_0 i}.
$$
Пользователю рекомендуются $k$ фильмов с наибольшими значениями $p_i$.


Реализуйте класс `ItemBased`, наследованный от `MemoryBased`.

Класс `ItemBased` должен иметь метод реализованный `recomendation_k` с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [0]:
class ItemBased(MemoryBased):
    def recomendation_k(self, user_vectors, k=20):
      """user_vectors - матрица размера N x M, где M - общее число фильмов, N - число пользователей, для которых делаем рекомендации"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass
    

### Задание 4 [1 балл]

Сформируйте матрицы $R = \{r_{ui}\}_{u \in U, \, i \in I}$ по данным `transactions_train` и `transactions_test`, где $r_{ui}$ — доля времени взаимодействовия пользователя с фильмом.

С помощью Item-Based и User-Based подходов получите предсказания для пользователей из тестовой выборки и оцените качество с помощью реализованной в задании 0 функции `mnap_k` при `k=20`.

### Задание 5 [3 балла]

#### Latent factor model (LFM)

В этом подходе значение целовой переменной $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $R^K$ латентных признаков:
$$
    r_{ui}
    \approx
    \langle p_u, q_i \rangle.
$$



Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы, и пользователей и в котором целевая переменная является мерой близости между фильмами и пользователями.

Для настройки модели будем минимизировать следующий функционал:

\begin{equation}
\label{eq:lfmReg}
    \sum_{(u, i) \in R}
        \left(
            r_{ui}
            - \langle p_u, q_i \rangle
        \right)^2
    +
    \lambda
    \sum_{u \in U}
        \|p_u\|^2
    +
    \mu
    \sum_{i \in I}
        \|q_i\|^2
    \to
    \min_{P, Q}
\end{equation}

В [статье](https://dl.acm.org/doi/10.1145/1864708.1864726) описан метод оптимизации ALS (Alternating Least Squares) для данного функционала.
В методе проводятся $N$ итераций, в рамках каждой итерации сначала оптимизируется $p$ при фиксированном
$q$, затем $q$ при фиксированном $p$.

Составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in R^{n_u×K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ – количество оценок пользователя $u$.
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке Ridge-регрессии и выглядит так:
$$A_u = Q[u]^T Q[u] $$
$$d_u = Q[u]^Tr_u $$
$$p_u = (\lambda n_uI + A_u)^{−1}d_u
$$

Выпишите формулы для перенастройки $q_i$ при фиксированной матрице $P$.

**Ответ:**

Реализуйте функцию `latent_factor`, которая для каждого пользователя из списка `test` возвращает top-k идентификаторов контента (`element_uid`).

Для тестирования матрицы `P` и `Q` задайте случайными `0.1 * np.random.random(...)`.

Исследуйте качество и время работы в зависимости от размерности $K$
пространства латентных признаков. Ведет ли увеличение $K$ к переобучению?


In [0]:
lambda_p = 0.2
mu = 0.001
N = 20
K = 10

In [0]:
train = transactions_train[['user_uid', 'element_uid']]
target = transactions_train['target']
test = transactions_test['user_uid'].unique()

In [0]:
def latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k = 20):
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・
    pass

In [0]:
train = pd.DataFrame({'user_uid': [1, 1, 2, 2], 'element_uid': [1, 2, 3, 4]})
target =  np.array( [0.1, 0.8, 0.2, 0.3 ])
test = np.array([1, 2])
lambda_p = 0.2
mu = 0.001
N = 20
K = 10
us = train.loc[:,'user_uid'].astype('int')
mov = train.loc[:,'element_uid'].astype('int')
U = np.unique(us)
I = np.unique(mov)
Q = 0.1 * np.ones((I.max(), K))
P = 0.1 * np.ones((U.max(), K))
assert np.all(latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k = 20) == np.array([[2], [2]]))

Оцените  качество разботы функции `latent_factor` на тестовой выборке по метрике `mnap_k` при `k=20`.

### Задание 6 [2 балла]

#### Content-based

В соревновании осталось много неиспользуемых нами данных. 
Сформируйте из них признаковое описание пары (пользователь, фильм), придумав минимум 10 содержательных (!) признаков. Целевой переменной, как и раньше, будет являться доля времени взаимодействия пользователя и фильма. Соберите данные, обучите на полученных данных линейную регрессию и градиентный бустинг. 
Для каждого пользователя из тестовой выборки предскажите 20 фильмов, с максимальной долей времени взаимодействия. Оцените качество предсказания с помощью функции `mnap_k` при `k=20`. Какого качество удалось достичь?

### Бонусное задание [3+ бонусных балла]

Используя любые данные из [конкурса](https://boosters.pro/championship/rekko_sand/overview), постройте предсказания для тестовой выборки из соревнования, отправьте свое решение и преодолейте в лидерборде порог `0.03`.

Подробно опишите ваше решение.

За преодоление порога вы можете получить от 3 баллов — конкретный балл будет зависеть от качества решения по сравнению с другими студентами, также выполнившими это задание.